In [1]:
import ee
import geemap
import os
import threading
from threading import Lock

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Define Area of Interest (AOI)
aoi = ee.Geometry.Polygon([
    [[-51.163395, -18.742186], [-51.163395, -18.534308],
     [-50.957458, -18.534308], [-50.957458, -18.742186],
     [-51.163395, -18.742186]]
])

# Sentinel-2 collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2020-01-01', '2020-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda image: image.clip(aoi))

# Select bands of interest and get the median image
bands = ['B2', 'B3', 'B4', 'B5', 'B8', 'B11']
image = sentinel2.select(bands).median()

# Get the date of the middle image in the collection
middle_date = sentinel2.sort('system:time_start').toList(sentinel2.size()).get(sentinel2.size().divide(2).floor()) 
middle_date = ee.Image(middle_date).date()

# Set the system:time_start property for the median image
image = image.set('system:time_start', middle_date.millis())

# Apply water mask using MNDWI index 
MNDWI = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
water_mask = MNDWI.gt(0.3)
image = image.updateMask(water_mask)

# Calculate NDCI (Normalized Difference Chlorophyll Index)
NDCI = image.normalizedDifference(['B5', 'B4']).rename('NDCI')

# Estimate chlorophyll concentration using NDCI
# This is a simple linear transformation and may need calibration for your specific water body
chlorophyll = NDCI.multiply(14.039).add(1.977).rename('chlorophyll_estimate')

# Apply the water mask to the chlorophyll estimate
chlorophyll = chlorophyll.updateMask(water_mask)

# Set the system:time_start property for the chlorophyll image
chlorophyll = chlorophyll.set('system:time_start', image.get('system:time_start'))

# Directory to save TIFF files
save_directory = 'analises_clorofila/cacu'  
os.makedirs(save_directory, exist_ok=True)

# Function to split AOI and export images in parallel
def split_aoi_and_export(aoi, n_tiles, scale, image, lock, tile_list):
    aoi_bounds = aoi.bounds().coordinates().getInfo()[0]
    xmin, ymin = aoi_bounds[0][0], aoi_bounds[0][1]
    xmax, ymax = aoi_bounds[2][0], aoi_bounds[2][1]
    x_step = (xmax - xmin) / n_tiles
    y_step = (ymax - ymin) / n_tiles

    def export_tile(i, j):
        x0 = xmin + i * x_step
        x1 = xmin + (i + 1) * x_step  
        y0 = ymin + j * y_step
        y1 = ymin + (j + 1) * y_step
        tile = ee.Geometry.Polygon([[[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]])
        tile_image = image.clip(tile)
        tile_list.append(tile_image)
        out_file = os.path.join(save_directory, f'EstimatedChlorophyll_Tile_{i+1}_{j+1}.tif')
        lock.acquire()
        try:
            geemap.ee_export_image(tile_image, filename=out_file, scale=scale, region=tile)
            print(f'Image saved locally: {out_file}')
        except Exception as e:
            print(f"Error exporting tile {i+1}_{j+1}: {str(e)}")
        finally:
            lock.release()
            
    threads = []
    for i in range(n_tiles):
        for j in range(n_tiles):
            t = threading.Thread(target=export_tile, args=(i, j))
            threads.append(t)
            t.start()

    for t in threads:
        t.join()

# Export tiles        
tile_list = []
n_tiles = 2  # NxN grid
lock = Lock()        
split_aoi_and_export(aoi, n_tiles, scale=30, image=chlorophyll, lock=lock, tile_list=tile_list)

# Merge tiles using mosaic
merged_image = ee.ImageCollection(tile_list).mosaic()

# Calculate min and max values of chlorophyll after merging
min_max_values = merged_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['chlorophyll_estimate_min']  
max_value = min_max_values['chlorophyll_estimate_max']

print(f"Minimum Estimated Chlorophyll Value: {min_value}")
print(f"Maximum Estimated Chlorophyll Value: {max_value}")

# Display the merged image on the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Visualization parameters
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        'blue', 'cyan', 'green', 'yellow', 'orange', 'red',  
        'darkred', 'purple', 'magenta', 'brown', 'black'
    ]
}
Map.addLayer(merged_image, vis_params, 'Estimated Chlorophyll')

# Add AOI boundary as a white outline with some transparency
aoi_style = {
    'color': 'white', 
    'width': 2,
    'fillColor': 'transparent'
}
Map.addLayer(aoi, aoi_style, 'AOI Boundary')

Map.addLayerControl()

# Function to add legend  
def add_legend(map_obj, title, palette, min_value, max_value):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>low</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>high</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

# Add legend to the map
add_legend(Map, 'Estimated Chlorophyll', vis_params['palette'], min_value, max_value)  

# Show the map
Map

Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_clorofila\cacu\EstimatedChlorophyll_Tile_1_1.tif
Image saved locally: analises_clorofila/cacu\EstimatedChlorophyll_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_clorofila\cacu\EstimatedChlorophyll_Tile_1_2.tif
Image saved locally: analises_clorofila/cacu\EstimatedChlorophyll_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_clorofila\cacu\EstimatedChlorophyll_Tile_2_1.tif
Image saved locally: analises_clorofila/cacu\EstimatedChlorophyll_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_clorofila\cacu\EstimatedChlorophyll_Tile_2_2.tif
Image saved locally: analises_clorofila/cacu\EstimatedChlorophyll_Tile_2_2.tif
Minimum Estimated Chlorophyll Value: -0.3748571428571428
Maximum Estimated Chlorophyll Value: 9.66886590038314


Map(center=[-18.63825382061626, -51.06042650000007], controls=(WidgetControl(options=['position', 'transparent…